What’s wrong?

Always returns zenith… list of what I think may be the issue. 

1. Could be due to indexing, but I did an equation for this, along with working on it today

2. Could be because I do not include the same conditionals I have in my slower version. 

3. The array of chi squared vals breaks somewhere, automatically returning zeros. Not sure why though.


I could resolve 2 by figuring out how to replace values in an array, need to be able to go into specific indices, and I’m not sure if there’s a way to do that without iterating, rendering this entire thing worthless. 



Thanks!


In [ ]:
import time as time
import burstutils

In [ ]:
chiResponse = [100,2,3,50]
chiseps = [0, np.pi/2, np.pi, 0]
chiResponse = [1e5 if x>np.pi/2 else x for x in chiseps]

chiResponse

In [ ]:
start = time.time()
a = np.zeros(100) 
a = [[1,1,1] if o==0 else o for o in a]
end = time.time()
A =(end-start)
print(A)

In [ ]:
a

In [ ]:
start = time.time()

normarr = [1,1,1]
normarrs = []
for garc in range(10):
    normarrs.append([normarr[0],normarr[1],normarr[2]])
end = time.time()
B =(end-start)
print(B)

In [ ]:
A

In [ ]:
B

To run through this stuff, just hit run on the cells in order. 

The first cell contains the class I use to generate GRBs

In [1]:

#Needs to know how to point to healpy 
import healpy as hp

class Sky():
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):

        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = hp.nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(hp.pix2ang(NSIDE,i))

    def say_Ao(self):
        """
        """

        print("The GRBs being tested will be " + str(self.Ao) + " counts strong.")
        

The next cell contains a class I call "SlowCube," these are my original simulations, depending on the iterative process, sacrificing speed for actually working. Note that they use the original function "solver," which is the function I use to calculate the minimum chi square and the corresponding sky position.

The following cell contains the "FastCube" class. This is the simulation I hope to use to be able to run quicker simulations. 

In [3]:
import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s
import time as time

class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
        
            self.tiltC = self.tiltA = dettilt
            self.tiltD = self.tiltB
        self.zenith = [0 , 0]
        self.bg = background

            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, test=False):   #is this how I inherit? 
        start = time.time()
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        if test:
            sample = 1
            samples = 1  #times  per sky pos
            nCtheta = 2   #over sky chi points
            nCphi = 2
            nCA = 2
            nFtheta = 2
            nFphi = 2
            nFA = 2
        else:
            sample = len(GRB.sourceangs) 
            samples = 5  #times  per sky pos
            nCtheta = 22   #over sky chi points
            nCphi = 54
            nCA = 40
            nFtheta = 100
            nFphi = 130
            nFA = 100
        self.localizationerrors = []    
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:
                sepA=bf.angle(sourcexyz,self.normA)
                   # print("separation from A is " + str(np.rad2deg(sepA)))
                   #this check passes.  
               
                if sepA < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryA=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normA))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryA = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsA = dtheoryA + self.bg #another artifact, incl this background effect somewhere
                unccountsA = np.sqrt(countsA)
                detactualA = rand.gauss(countsA,unccountsA)  #there is a lot of noise, present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = detactualA
                
                sepB=bf.angle(sourcexyz,self.normB)
                   # print("separation from B is " + str(np.rad2deg(sepB)))
                   #this check passes.  
               
                if sepB < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryB=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normB))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryB = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsB = dtheoryB + self.bg #another artifact, incl this background effect somewhere
                unccountsB = np.sqrt(countsB)
                detactualB = rand.gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = detactualB

                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation from C is " + str(np.rad2deg(sepC)))
                   #this check passes.  
               
                if sepC < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryC=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normC))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryC = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = np.sqrt(countsC)
                detactualC = rand.gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = detactualC
                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation from D is " + str(np.rad2deg(sepD)))
                   #this check passes.  
               
                if sepD < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryD=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normD))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryD = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = np.sqrt(countsD)
                detactualD = rand.gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = detactualD
                
                #coarse to fine optimization
                CchiA = bf.quad_solver(detcountsA,self.normA,0,90,0,360,0,1000,nCtheta,nCphi,nCA,self.bg)
                CchiB = bf.quad_solver(detcountsB,self.normB,0,90,0,360,0,1000,nCtheta,nCphi,nCA,self.bg)
                CchiC = bf.quad_solver(detcountsC,self.normC,0,90,0,360,0,1000,nCtheta,nCphi,nCA,self.bg)
                CchiD = bf.quad_solver(detcountsD,self.normD,0,90,0,360,0,1000,nCtheta,nCphi,nCA,self.bg)
                
                Cchisquared = np.add(np.add(CchiA,CchiB),np.add(CchiC,CchiD)) #adds it all up for total chi2
                Cchisquareds=list(Cchisquared)
                chisquareds = Cchisquareds
                chimin = min(chisquareds)
                oa = np.linspace(0,90,nCtheta)
                ob = np.linspace(0,360,nCphi)
                Aofit = np.linspace(0,1000,nCA)
                thetaloc = (oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
                philoc = (ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
               # print(thetaloc,philoc)
                recvec = hp.ang2vec(np.deg2rad(thetaloc),np.deg2rad(philoc))
                print("Obtained offset of " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg. ")

                loop +=1

In [ ]:
                
                
                
                Cthetaloc,Cphiloc,CAoguess = bf.indexer(Cchisquareds,0,90,0,360,0,1000,nCtheta,nCphi,nCA)
               # print("Ctheta: " + str(Cthetaloc))
              # What if I only do it with Coarse result since fine is fucking it up?
                recvec = hp.ang2vec(np.deg2rad(Cthetaloc),np.deg2rad(Cphiloc))

                locunc.append(bf.angle(sourcexyz,recvec))
                print("Obtained offset of " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg. ")
                loop +=1
            print("avg loc unc  " +  str(s.mean(np.rad2deg(locunc))))
            

In [5]:
GRBtester = Sky(8,500)  #for both the fast and slow simulation, going to use an NSIDE=8 and strength =500 

In [4]:
cube2 = FastCube(1000,45)  #same things

In [6]:
cube2.response2GRB(GRBtester)

Testing [ 5.85026706 45.        ]
Obtained offset of 18.60746596943819 deg. 
Obtained offset of 13.512008037606485 deg. 
Obtained offset of 14.006209458553398 deg. 
Obtained offset of 14.219266958457682 deg. 
Obtained offset of 18.57909376347688 deg. 
Testing [  5.85026706 135.        ]
Obtained offset of 13.954658267875342 deg. 
Obtained offset of 18.703406249039922 deg. 
Obtained offset of 13.954658267875342 deg. 
Obtained offset of 17.48187637449545 deg. 
Obtained offset of 27.05438208681405 deg. 
Testing [  5.85026706 225.        ]
Obtained offset of 18.167006109916674 deg. 
Obtained offset of 22.642536706015363 deg. 
Obtained offset of 10.717489610849402 deg. 
Obtained offset of 14.14302940483158 deg. 
Obtained offset of 22.33830907614082 deg. 
Testing [  5.85026706 315.        ]
Obtained offset of 13.782355513704205 deg. 
Obtained offset of 14.219266958457682 deg. 
Obtained offset of 18.226549532690246 deg. 
Obtained offset of 13.583896104439196 deg. 
Obtained offset of 18.685618

KeyboardInterrupt: 

In [ ]:
                delta = (5)
                FchiA = bf.quad_solver(detcountsA,self.normA,Cthetaloc-delta,Cthetaloc+delta,Cphiloc-delta,Cphiloc+delta,CAoguess-delta,CAoguess+10,nFtheta,nFphi,nFA,self.bg)
                FchiB = bf.quad_solver(detcountsB,self.normB,Cthetaloc-delta,Cthetaloc+delta,Cphiloc-delta,Cphiloc+delta,CAoguess-delta,CAoguess+10,nFtheta,nFphi,nFA,self.bg)
                FchiC = bf.quad_solver(detcountsC,self.normC,Cthetaloc-delta,Cthetaloc+delta,Cphiloc-delta,Cphiloc+delta,CAoguess-delta,CAoguess+10,nFtheta,nFphi,nFA,self.bg)
                FchiD = bf.quad_solver(detcountsD,self.normD,Cthetaloc-delta,Cthetaloc+delta,Cphiloc-delta,Cphiloc+delta,CAoguess-delta,CAoguess+10,nFtheta,nFphi,nFA,self.bg)
                
                Fchisquared = np.add(np.add(FchiA,FchiB),np.add(FchiC,FchiD)) #adds it all up for total chi2
                Fchisquareds=list(Fchisquared)
                
                Fthetaloc,Fphiloc,FAoguess = bf.indexer(Fchisquareds,Cthetaloc-delta,Cthetaloc+delta,Cphiloc-delta,Cphiloc+delta,CAoguess-10,CAoguess+10,nFtheta,nFphi,nFA)
                
                if Fthetaloc > 180:
                   # print("it recovered an unrealistic answer, skip")
                    Fthetaloc = 180
                elif Fthetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    Fthetaloc = 0
                
                """Aftr getting the minimum value in the chisquareds array, the commented out functions below are supposed to find where in the theta,phi, and A arrays what this corresponds to. It doesn't. 
                """
             
               
              #  print("Recovered position at " + str(recpos))
                recvec = hp.ang2vec(np.deg2rad(Fthetaloc),np.deg2rad(Fphiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
                loop +=1
            print("avg loc unc  " +  str(s.mean(np.rad2deg(locunc))))
            self.localizationerrors.append(s.mean(np.rad2deg(locunc)))
        end = time.time()
        print("Time elapsed for quicker: " + str(end-start))
        return self.localizationerrors

The most important part of these classes is the "response2GRB" function. As you'll see in both, they actually happen to print out different results. There's a lot more they can do too, but for now I feel it best demonstrates what they're supposed to do by having the working one print out the localization uncertainty at each sky position, and the broken one print out where it thinks in the big array of possible values what the localized position is. For "FastCube" This is typically 2, which would correspond to something at the zenith, which would be incorrect.


To see what I'm talking about, simply run the cells below:

-They both correspond to BurstCube models where all detectors are evenly spaced and tilted down by 45°, with a background of 1000 counts. 

In [ ]:
cube1 = SlowCube(1000,45)  #the first burstcube sim is here
cube1.response2GRB(GRBtester, test=True)  #just run this cell, you'll see that as different positions in the sky are tested,
# you'll see that the average localizations of a source is often <6°. 

In [ ]:
oa = np.linspace(0,90,20)
ob = np.linspace(0,360,20)
Aofit = np.linspace(0,1000,20)

In [ ]:
thetaloc = np.rad2deg(oa[int((chisquareds[449]-(chisquareds[449] % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])

In [ ]:
chisquareds = np.linspace(0,100,20*20*20)

At this point I'm more concerned with what is being printed out 

In [ ]:
thetaloc

In [ ]:
np.min(stuff[3])

In [ ]:
np.min?

In [ ]:
thetas = np.linspace(0,90,2)
phis = np.linspace(0,360,2)

In [ ]:
xx,yy = np.meshgrid(phis,thetas)

In [ ]:
np.concatenate(xx)

In [ ]:
np.concatenate(yy)

In [ ]:
hp.vec2ang(hp.ang2vec(np.deg2rad(np.concatenate(yy)),np.deg2rad(np.concatenate(xx))))   #check holds in this meshgrid. 

In [ ]:
theta = np.deg2rad(np.linspace(0,45,2))
phi = np.deg2rad(np.linspace(0,90,2))
mphi,mtheta = np.meshgrid(phi,theta)
allthetas = np.concatenate(mtheta)
allphis = np.concatenate(mphi)
allvecs = hp.ang2vec(allthetas,allphis)
As= np.linspace(0,1000,2)
normarr = [0,1/np.sqrt(2),1/np.sqrt(2)]
normarrs = []
for garc in range((len(theta)*len(phi))):
    normarrs.append([normarr[0],normarr[1],normarr[2]])
        
seps = bf.findAngles(allvecs,normarrs)

In [ ]:
np.rad2deg(seps)

In [ ]:
np.linspace(0,180,2)

In [ ]:
import astropy

In [ ]:
x+x

In [ ]:
x

In [ ]:
np.divide(x,x)

In [ ]:
np.power(x,3)

In [ ]:
np.divide(np.power(np.subtract(chiResponse,detval),2),detval)

In [ ]:
theta = np.deg2rad(np.linspace(0,90,2))
phi = np.deg2rad(np.linspace(0,180,2))
mphi,mtheta = np.meshgrid(phi,theta)
allthetas = np.concatenate(mtheta)
allphis = np.concatenate(mphi)
allvecs = hp.ang2vec(allthetas,allphis)

In [ ]:
allvecs

In [ ]:
normarr = cube2.normA
normarrs = []
for garc in range((len(theta)*len(phi))):
    normarrs.append([normarr[0],normarr[1],normarr[2]])

In [ ]:
normarrs

In [ ]:
np.rad2deg(bf.findAngles(allvecs,normarrs))

In [ ]:
def chiresponse(A):
    """
    Meant to imitate the actual response of a scintillator.
    Inputs 2 vectors, and responds with a cos^x dependence.
    """
    #meant to imitate the response of the detectors for effective area vs. angle, found to be around .77
 #   print(length(A),length(B))
#if cosine is negative, 

    mask = A > np.pi/2.

    A[mask] = 0
    A[~mask] = pow(abs(np.cos(A[~mask])),0.76)
    
    
    return A

In [ ]:
x = np.array([1000,1001,2030])

In [ ]:
[0 if a==1000 else a for a in x]

In [ ]:
def indexer(bottheta,toptheta,botphi,topphi,botA,topA,n):
    oa = np.deg2rad(np.linspace(bottheta,toptheta,n))
    ob = np.deg2rad(np.linspace(botphi,topphi,n))
    Aofit = np.deg2rad(np.linspace(botA,topA,n))
    thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
    philoc = np.rad2deg(ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
    Aoguess=Aofit[int((chisquareds.index(chimin) % (len(ob)*len(Aofit)))  % len(Aofit))]
    
    return thetaloc, philoc, Aoguess

In [ ]:
import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s
import time as time
class SlowCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
        
            self.tiltC = self.tiltA = dettilt
            self.tiltD = self.tiltB
        self.zenith = [0 , 0]
        self.bg = background

            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, test = False):   #is this how I inherit? 
        start = time.time()
        #first need to include the GRB.
           
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        if test:
            sample = 1
            samples = 1
        else:
            sample = len(GRB.sourceangs)
            samples = 50
        self.localizationerrors = []    
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:  #how often a sky position is sampled, only 1 for now, the larger it is though, the slower the sim. 
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=GRB.Ao*bf.response(bf.angle(sourcexyz,self.dets[j]))  #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(counts)
                
                
                self.detcounts = detcounts
                
                finethetaloc,finephiloc,fineAo, finechis = bf.solver(detcounts,self.dets,0,90,0,360,50,50,self.bg)
                
               # print(coarsechis)
               # print(finechis)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                recpos = [finethetaloc,finephiloc]
              #  print("Recovered position at " + str(recpos))
                
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
              #   print("loc unc: " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg")
                loop+=1
          #  print("Obtained avg loc unc of " + np.rad2deg(s.mean(locunc)) + "")
            if len(locunc)>0:
                self.localizationerrors.append(np.rad2deg(s.mean(locunc)))
            else:
                self.localizationerrors.append(1000)
                #some big # signifying that it can't catch it 

            print("avg loc unc  " +  str(s.mean(np.rad2deg(locunc))))
        end = time.time()
        print("Time elapsed slower: " + str(end-start))
        return self.localizationerrors
